In [1]:
%reload_ext autoreload


In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import numpy as np
from typing import Generator, Any
import pandas as pd
import json
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os
from rich import print as rprint
from mbay_nmt.utils import domain as d
from mbay_nmt.utils.models import new_object_id
from datasets import load_dataset, Dataset, DatasetDict
from rich import print as rprint

load_dotenv()

True

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

## Huggingface datasets creation


In [4]:
from typing import Literal, TypedDict


class Record(TypedDict):
    id: str
    entry_id: str
    type: Literal["entry", "example", "expression"]
    mbay: str
    french: str
    english: str

In [4]:
CSV_DATASET_PATH = "../../datasets/mbay-translations-flattened.csv.gzip"
SPLIT_DATASET_PATH = "../../datasets/mbay-translations/"
TOKENIZED_DATASET_PATH = "../../datasets/mbay-translations-tokenized/"

In [27]:
# df = pd.DataFrame(records)
# df.to_csv(CSV_REVIEW_DATASET_PATH, index=False)

We are currently at row 72


In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Use the JSON key file you downloaded when you created your Service Account
json_key_file = "/Users/nasoungadoy/.googlecloud/fluid-mind-303321-602c5b57c4e7.json"

# Define the scope
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# Load the credentials and create a client to interact with the Google Drive API
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key_file, scope)
google_sheets = gspread.authorize(credentials)

In [7]:
sheet = google_sheets.open_by_url(
    "https://docs.google.com/spreadsheets/d/1KbpQS0RdcLhUOJNFNdXlnTDZBKEU2S8Q7DnJhPYrr2M"
).sheet1
columns = sheet.row_values(1)
columns = {columns[i]: i + 1 for i in range(len(columns))}
columns

{'id': 1,
 'entry_id': 2,
 'type': 3,
 'mbay': 4,
 'french': 5,
 'english': 6,
 'flagged': 7,
 'comment': 8}

In [8]:
records = sheet.get_all_records()

In [9]:
df = pd.DataFrame(records)
df = df[df["flagged"] != "TRUE"].drop(columns=["flagged", "comment"])

In [53]:
df

,id,entry_id,type,mbay,french,english
0,64eca312f6197fd20d762cf5,,entry,"àlmbétɨ̀, àlmétɨ̀",allumette.,match.
1,64eca312f6197fd20d76096f,64eca312f6197fd20d762cf5,example,gà àlmbétɨ̀,allumer une allumette,light a match
2,64eca312f6197fd20d760970,64eca312f6197fd20d762cf5,example,ī-gá àlmbétɨ̀ ādɨ̄-m̄.,Allumez une allumette pour moi.,Light a match for me.
3,64eca312f6197fd20d760971,64eca312f6197fd20d762cf5,example,kùm-àlmbétɨ̀,allumette non allumée,unlit match stick
4,64eca312f6197fd20d760972,64eca312f6197fd20d762cf5,example,Màn̄ à ɔ̀dɨ̀ kùm-àlmbétɨ̀ ànḛ̄ à ùnjɨ̄ àĺ.,"Si l'eau touche une allumette, elle ne s'allum...",If water touches a matchstick it won't light.
...,...,...,...,...,...,...
10724,64fc50f7286b18ef7de0f25a,64fc50f7286b18ef7de0f259,example,m̄-hɔ́-ī àbàsúl kɨ̀tɔ̄-kàdɨ̄ ī-sá jī-í kɨ́rā àĺ.,"Je t'ai battu à plate couture, car tu n'as pas...","I skunked you, since you didn't take a single ..."
10725,64fc50f7286b18ef7de0f25b,,entry,à,ensuite; et donc; et ensuite,then; and so; and then
10726,64fc50f7286b18ef7de0f25c,64fc50f7286b18ef7de0f25b,example,Dèē-kɨ́-dḛ̀ḛ́ nā̰a̰ tá̰a̰ nèl̄ tà-á péléng à n...,La femme a goûté la sauce et elle l'a tellemen...,The woman tasted the sauce and she liked it so...
10727,64fc50f7286b18ef7de0f25d,,entry,àbɨ̀lày,fouet fait de cuir {traditionnellement en peau...,whip made of leather {traditionally hippopotam...


In [57]:
df.drop_duplicates(subset=["english"]).shape

(10075, 6)

In [21]:
df.to_csv(CSV_DATASET_PATH, index=False, compression="gzip")

In [5]:
df = pd.read_csv(CSV_DATASET_PATH, compression="gzip")

In [6]:
dst = Dataset.from_pandas(df)
# dst = dst.remove_columns(["__index_level_0__"])
dst

Dataset({
    features: ['id', 'entry_id', 'type', 'mbay', 'french', 'english'],
    num_rows: 10726
})

In [19]:
from transformers import AutoTokenizer
from mbay_nmt.fine_tune_t5.utils import preprocess_records, format_prompt_output_pairs

t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")

/Users/nasoungadoy/Code/mbay-translator/py/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Final check


In [ ]:
dst[10]

{'id': '64eca312f6197fd20d76098e',
 'entry_id': '64eca312f6197fd20d762d04',
 'type': 'example',
 'mbay': 'ī-ɗāa àngérì nà̰ wétɨ́ ī-sō hólēe tɨ́ nò.',
 'french': 'Faites attention de ne pas tomber dans le trou.',
 'english': 'Be careful lest you fall into the hole.'}

In [8]:
format_prompt_output_pairs(dst[10:11])

NameError: name 'format_prompt_output_pairs' is not defined

In [11]:
dst_pairs = dst.map(
    format_prompt_output_pairs, batched=True, remove_columns=dst.column_names
)

Map:   0%|          | 0/10726 [00:00<?, ? examples/s]

In [13]:
import random

In [14]:
random.choices(range(len(dst_pairs)), k=10)

[37644, 8011, 18099, 31062, 41935, 11592, 37806, 20706, 23798, 12510]

In [15]:
sample_ids = random.choices(range(len(dst_pairs)), k=10)
dst_pairs[sample_ids]

{'inputs': ['Translate English to Mbay: soon, in a little while',
  'Translate English to Mbay: thoroughly (shrink, thin).',
  "Translate English to Mbay: go announce [s.o.'s] death",
  'Translate English to Mbay: maternal cousin',
  'Translate Mbay to French: kàdɨ̄',
  'Translate English to Mbay: A tradition is a custom of the people.',
  'Translate Mbay to French: hōŕ gìndɨ̄',
  'Translate Mbay to French: ngè-ndò̰o̰ à èl-m̄ tà kɨ́-dà mò̰y tɨ́ lò-ḿ.',
  'Translate French to Mbay: Ils ont mis des entraves aux chevilles du cheval.',
  'Translate Mbay to French: Dèē-kɨ́-dḛ̀ḛ́ kɨ́ ngàw kòy ndūr pútɨ́-pútɨ́.'],
 'targets': ['sḭ́ḭ-kɨ́-nòó-tɨ́',
  'mbḭ́yá̰',
  'àw̄ kɨ̀là-yòo lò [dèē]',
  'ngōn-nān',
  'donner',
  'Yá̰a̰ kōo-wēe ì yá̰a̰ gír lò dèē.',
  'morceau de tuyau en métal',
  'La voyante va me parler de ma maladie (expliquer sa cause).',
  'Lā-n̄ dálā kɔ́ɔ́-njà síndá tɨ́.',
  'La femme dont le mari est décédé se roula énergiquement dans le chagrin.']}

In [21]:
from functools import partial

tokenized_dst_check = dst.map(
    partial(preprocess_records, t5_tokenizer),
    batched=True,
    remove_columns=dst.column_names,
)

Map:   0%|          | 0/10726 [00:00<?, ? examples/s]

In [27]:
tokenized_dst_check

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 42904
})

In [66]:
def detokenize_batch(tokenizer: AutoTokenizer, batch: dict[str, list[list[int]]]):
    input_ids = batch["input_ids"]
    input_ids = np.where(input_ids != -100, input_ids, tokenizer.pad_token_id)
    decoded_input_ids = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

    # labels = batch["labels"]
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return {
        "inputs": decoded_input_ids,
        # "labels": decoded_labels,
    }

In [67]:
detokenized_dst_check = tokenized_dst_check.map(
    partial(detokenize_batch, t5_tokenizer),
    batched=True,
    # remove_columns=dst.column_names,
)

Map:   0%|          | 0/42904 [00:00<?, ? examples/s]

In [69]:
detokenized_dst_check[sample_ids]["inputs"]

['Translate English to Mbay: soon, in a little while',
 'Translate English to Mbay: thoroughly (shrink, thin).',
 "Translate English to Mbay: go announce [s.o.'s] death",
 'Translate English to Mbay: maternal cousin',
 'Translate Mbay to French: kàdɨ̄',
 'Translate English to Mbay: A tradition is a custom of the people.',
 'Translate Mbay to French: hōŕ gìndɨ̄',
 'Translate Mbay to French: ngè-ndò̰o̰ à èl-m̄ tà kɨ́-dà mò̰y tɨ́ lò-ḿ.',
 'Translate French to Mbay: Ils ont mis des entraves aux chevilles du cheval.',
 'Translate Mbay to French: Dèē-kɨ́-dḛ̀ḛ́ kɨ́ ngàw kòy ndūr pútɨ́-pútɨ́.']

### Final dataset


In [43]:
train_test = dst.train_test_split(0.2)
test_valid = train_test["test"].train_test_split(0.5)

train_test_valid_dst = DatasetDict(
    {
        "train": train_test["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"],
    }
)
train_test_valid_dst

DatasetDict({
    train: Dataset({
        features: ['id', 'entry_id', 'type', 'mbay', 'french', 'english'],
        num_rows: 8580
    })
    test: Dataset({
        features: ['id', 'entry_id', 'type', 'mbay', 'french', 'english'],
        num_rows: 1073
    })
    validation: Dataset({
        features: ['id', 'entry_id', 'type', 'mbay', 'french', 'english'],
        num_rows: 1073
    })
})

In [44]:
train_test_valid_dst["test"][2]

{'id': '64fc50f7286b18ef7de0bfb5',
 'entry_id': '',
 'type': 'entry',
 'mbay': 'ɓà-kùm-dàm-á',
 'french': 'personne aux grands yeux',
 'english': 'person with big eyes'}

In [45]:
train_test_valid_dst.save_to_disk(SPLIT_DATASET_PATH)

Saving the dataset (0/1 shards):   0%|          | 0/8580 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1073 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1073 [00:00<?, ? examples/s]

In [10]:
# T5Tokenizer.__call__?

In [17]:
from transformers import AutoTokenizer

t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/nasoungadoy/Code/mbay-translator/py/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [48]:
train_test_valid_dst.column_names["train"]

['id', 'entry_id', 'type', 'mbay', 'french', 'english']

In [18]:
test_tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

In [49]:
input_txt = (
    f"Translate French to Mbay: Récemment, les citrons sont très difficiles à trouver."
)
target_txt = "Ngɔ̀r làā làmíǹ gásɨ̀ ngá̰y."

In [51]:
from functools import partial

final_dst = train_test_valid_dst.map(
    partial(preprocess_records, t5_tokenizer),
    batched=True,
    remove_columns=train_test_valid_dst["train"].column_names,
)

Map:   0%|          | 0/8580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1073 [00:00<?, ? examples/s]

Map:   0%|          | 0/1073 [00:00<?, ? examples/s]

In [52]:
final_dst.save_to_disk(TOKENIZED_DATASET_PATH)

Saving the dataset (0/1 shards):   0%|          | 0/34320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4292 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4292 [00:00<?, ? examples/s]